In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Esse notebook tem o objetivo de implementar o TSMOM do  Moskowitz. Portanto utilizarei uma parte do dataset, para facilitar a implementação. No dataset estão presentes apenas os preços de fechamento, e apenas de 1999 a 2019

In [2]:
# Importação do csv previamente limpo
df = pd.read_csv ("close_clean_data.csv")

In [3]:
df.head()

,date,AN_RAD.CSV,BN_RAD.CSV,CC_RAD.CSV,CN_RAD.CSV,CT_RAD.CSV,DA_RAD.CSV,DX_RAD.CSV,EC_RAD.CSV,EN_RAD.CSV,...,ZU_RAD.CSV,ZW_RAD.CSV,ZZ_RAD.CSV,DT_RAD.CSV,HS_RAD.CSV,LX_RAD.CSV,NK_RAD.CSV,SP_RAD.CSV,UB_RAD.CSV,AX_RAD.CSV
0,1999-06-21,42.81,143.90,2468.0,67.537,220.54,20.06,112.848,88.1480,3054.57,...,72.14,4614.72,163.364,75.10,11237.36,5618.6,16474.0,1464.92,82.26,7704.9
1,1999-06-22,43.12,143.72,2421.0,67.537,222.69,20.20,113.002,88.1247,2984.70,...,71.50,4536.94,166.007,74.87,11138.79,5586.4,16387.0,1453.39,82.11,7702.8
2,1999-06-23,43.39,142.89,2304.0,67.507,220.19,20.26,113.057,88.1060,3006.66,...,74.32,4558.54,165.846,74.47,11119.07,5536.9,16262.0,1445.85,81.88,7601.4
3,1999-06-24,43.42,143.63,2227.0,67.497,222.81,20.29,112.419,88.0781,2952.09,...,73.67,4528.30,162.323,73.86,10914.04,5466.2,16225.0,1435.62,81.59,7486.8
4,1999-06-25,43.64,143.52,2221.0,67.824,221.30,20.42,112.144,88.0781,2948.10,...,74.08,4562.87,162.963,74.17,10921.93,5478.0,16229.0,1431.52,81.66,7511.3


In [4]:
# Calculo dos retornos percentuais e indexacao da data
returns_daily = df.drop('date', axis=1).pct_change().drop(0)
returns_daily.index = pd.to_datetime(df['date'], format='%Y-%m-%d').drop(0)

returns_daily.tail()

,AN_RAD.CSV,BN_RAD.CSV,CC_RAD.CSV,CN_RAD.CSV,CT_RAD.CSV,DA_RAD.CSV,DX_RAD.CSV,EC_RAD.CSV,EN_RAD.CSV,ES_RAD.CSV,...,ZU_RAD.CSV,ZW_RAD.CSV,ZZ_RAD.CSV,DT_RAD.CSV,HS_RAD.CSV,LX_RAD.CSV,NK_RAD.CSV,SP_RAD.CSV,UB_RAD.CSV,AX_RAD.CSV
date,,,,,,,,,,,,,,,,,,,,,
2019-06-17,-0.002034,-0.003561,-0.006811,0.000201,0.010190,0.000000,-0.000155,-0.000255,0.007529,0.000518,...,-0.011370,0.001857,0.020897,-0.001048,0.003796,0.002337,0.003807,0.000518,0.000075,-0.001488
2019-06-18,0.003348,0.001588,0.008471,0.002477,0.006022,0.001188,0.001082,0.000000,0.014845,0.010358,...,0.037186,-0.014829,-0.018964,0.006877,0.010899,0.012821,0.005452,0.010324,0.003728,0.019823
2019-06-19,0.002322,0.008564,0.005200,0.006812,0.005088,0.007117,-0.005918,0.000970,0.003714,0.002478,...,-0.002587,-0.017404,0.001841,-0.001331,0.025634,-0.007379,0.004715,0.002461,-0.001189,-0.001907
2019-06-20,0.004632,0.002280,-0.026661,0.007297,-0.017868,0.011779,-0.004556,0.000510,0.009186,0.009034,...,0.057439,0.008138,-0.029096,0.000348,0.013229,0.004365,0.007039,0.009068,-0.000149,0.005163
2019-06-21,0.000576,0.002902,0.022895,-0.001251,-0.006064,-0.005239,-0.004369,-0.000306,-0.002734,-0.003209,...,0.006308,-0.000950,-0.037855,-0.002955,-0.002929,-0.006179,-0.012582,-0.003176,-0.001265,-0.007402


In [5]:
# Calculo dos desvio, e os pesos dos tickers no portifolio
std = returns_daily.std()
weights = std / sum(std)

In [6]:
# Construção da matrix com os retornos mensais
returns_monthly = returns_daily.resample('BM').last().ffill()

In [7]:
# definindo a função que calcula o momento IMPLEMENTACAO AINDA ESTA INCORRETA
#https://breakingdownfinance.com/finance-topics/alternative-investments/time-series-momentum/
def momentum (end, data, window):
    train = data[:str(end)]

    multplier = np.ones(shape = data.shape[1])
    for i in range(window):
        multplier = multplier * train.iloc[-i:1-i]
        
    return multplier

In [8]:
momentum (end='2018-12', data=returns_monthly, window=12).tail(13)

,AN_RAD.CSV,BN_RAD.CSV,CC_RAD.CSV,CN_RAD.CSV,CT_RAD.CSV,DA_RAD.CSV,DX_RAD.CSV,EC_RAD.CSV,EN_RAD.CSV,ES_RAD.CSV,...,ZU_RAD.CSV,ZW_RAD.CSV,ZZ_RAD.CSV,DT_RAD.CSV,HS_RAD.CSV,LX_RAD.CSV,NK_RAD.CSV,SP_RAD.CSV,UB_RAD.CSV,AX_RAD.CSV
date,,,,,,,,,,,,,,,,,,,,,
1999-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
returns_monthly.iloc[-1:]

,AN_RAD.CSV,BN_RAD.CSV,CC_RAD.CSV,CN_RAD.CSV,CT_RAD.CSV,DA_RAD.CSV,DX_RAD.CSV,EC_RAD.CSV,EN_RAD.CSV,ES_RAD.CSV,...,ZU_RAD.CSV,ZW_RAD.CSV,ZZ_RAD.CSV,DT_RAD.CSV,HS_RAD.CSV,LX_RAD.CSV,NK_RAD.CSV,SP_RAD.CSV,UB_RAD.CSV,AX_RAD.CSV
date,,,,,,,,,,,,,,,,,,,,,
2019-06-28,0.000576,0.002902,0.022895,-0.001251,-0.006064,-0.005239,-0.004369,-0.000306,-0.002734,-0.003209,...,0.006308,-0.00095,-0.037855,-0.002955,-0.002929,-0.006179,-0.012582,-0.003176,-0.001265,-0.007402


In [10]:
def sharpe_ratio(data, risk_free = 0):
    mean = data.mean()
    std = data.std()
    
    return (mean - risk_free) / std